# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,-33.73,74,20,3.09,CA,1704826544
1,1,yuzhno-kurilsk,44.0306,145.8556,-0.90,77,100,4.87,RU,1704826625
2,2,ribeira grande,38.5167,-28.7000,18.22,88,75,13.41,PT,1704826625
3,3,iskateley,67.6803,53.1512,-21.10,100,100,2.24,RU,1704826626
4,4,adamstown,-25.0660,-130.1015,23.98,88,99,4.08,PN,1704826626


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points('Lng','Lat',geo=True, tiles='OSM', frame_width=800, frame_height=600, size='Humidity', color='City',alpha=.75)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
temp_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
temp_data_df = temp_data_df.loc[(temp_data_df["Wind Speed"] < 4.5)]
temp_data_df = temp_data_df.loc[(temp_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
temp_data_df = temp_data_df.dropna()

# Display sample data
print(temp_data_df.shape)
temp_data_df.head()

(6, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,vallenar,-28.5708,-70.7581,26.99,30,0,3.81,CL,1704826626
212,212,pisco,-13.7000,-76.2167,26.03,65,0,3.60,PE,1704826693
219,219,birao,10.2849,22.7882,24.65,16,0,1.83,CF,1704826697
330,330,bousso,10.4795,16.7117,26.38,12,0,3.00,TD,1704826736
419,419,moyale,3.5167,39.0584,23.27,38,0,3.76,KE,1704826768


### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
city = temp_data_df["City"]
country = temp_data_df["Country"]
lat = temp_data_df["Lat"]
lng = temp_data_df["Lng"]
humidity = temp_data_df["Humidity"]

In [11]:
hotel_dict_df = pd.DataFrame({"City": city,
                        "Country": country,
                        "Lat": lat,
                        "Lng": lng,
                        "Humidity": humidity})

hotel_dict_df


,City,Country,Lat,Lng,Humidity
5,vallenar,CL,-28.5708,-70.7581,30
212,pisco,PE,-13.7000,-76.2167,65
219,birao,CF,10.2849,22.7882,16
330,bousso,TD,10.4795,16.7117,12
419,moyale,KE,3.5167,39.0584,38
450,ormara,PK,25.2088,64.6357,47


In [12]:
hotel_columns = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
temp_data_df[hotel_columns]

,City,Country,Lat,Lng,Humidity
5,vallenar,CL,-28.5708,-70.7581,30
212,pisco,PE,-13.7000,-76.2167,65
219,birao,CF,10.2849,22.7882,16
330,bousso,TD,10.4795,16.7117,12
419,moyale,KE,3.5167,39.0584,38
450,ormara,PK,25.2088,64.6357,47


In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_columns = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = pd.DataFrame(temp_data_df[hotel_columns])
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,vallenar,CL,-28.5708,-70.7581,30,
212,pisco,PE,-13.7000,-76.2167,65,
219,birao,CF,10.2849,22.7882,16,
330,bousso,TD,10.4795,16.7117,12,
419,moyale,KE,3.5167,39.0584,38,
450,ormara,PK,25.2088,64.6357,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000
hotel_params = {
    'apiKey': geoapify_key,
    'categories': 'accommodation.hotel',
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    hotel_params["filter"] = f'circle:{lon},{lat},{radius}'
    hotel_params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=hotel_params)
    
    # Convert the API JSON to Python format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vallenar - nearest hotel: Hotel del Marqués
pisco - nearest hotel: La Portada
birao - nearest hotel: No hotel found
bousso - nearest hotel: No hotel found
moyale - nearest hotel: Paul’s guest house
ormara - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
5,vallenar,CL,-28.5708,-70.7581,30,Hotel del Marqués
212,pisco,PE,-13.7000,-76.2167,65,La Portada
219,birao,CF,10.2849,22.7882,16,No hotel found
330,bousso,TD,10.4795,16.7117,12,No hotel found
419,moyale,KE,3.5167,39.0584,38,Paul’s guest house
450,ormara,PK,25.2088,64.6357,47,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)